### Before start we need:

Installation:
- Docker
- MySQLWorkbench
- Jupyter Notebook

Prerequisites:
- Be able to run Docker
- MySQL
- Some Python

### Your Toolkits to Prepare for SQL Interview

After this session, you will be able to
- host a standalone MySQL database for practicing SQL.
- create a (fake table) that you need for your practicing.
- have a rough idea what a SQL window function, a popular kind interview questions nowadays, is doing.

### Host Your MySQL Database

We will be demonstrating with MySQL. Because we are to play with window functions, we will need to use MySQL 8. If you have a MySQL 5.x (as what we have in our Docker image for our regular SQL class), unfortunately, it doesn't support window functions.

Here we would like to show you the fastest way we know to host a database, create tables you need, and start tackling your SQL questions. We will still use Docker but we are going to pull the <a href="https://hub.docker.com/_/mysql" target="_blank">official images</a>. The documentations on their web page shows you a variety of functionalities they provide us, including change the configurations and upload old SQL data. It's nice to read through them but today we only need the simplest Docker command:

```bash
docker run --name <some-mysql> -p <port number>:3306 -e MYSQL_ROOT_PASSWORD=<my-secret-pw> -d mysql:8
```

The only two things you need to decide (as you can see from the pretty self-explanatory arguments):
- the name of the container.
- the password for root.

The name allows you to find the container easily, you don't need to always refer to the container id if you have given a name to the container. The password is for security reason so if you don't allow connections from outside of your computer - that means if you don't change the default configuration - it's really not important. A simple password is fine.

The only thing you might need to pay more attention to is the port number because we need it for the connection from MySQLWorkbench. On the left hand side of the argument `-p`, most people would just use 3306 which is the same as the right hand side. In case your 3306 is allocated alreay, you need to try another port. If you have a local MySQL 5.x on you native system then you very likely need to use a port other than 3306. Try to start a container with the simple command below (if you don't insist to use your own name and password.) If you don't see any error message than you are fine:

```bash
docker run --name ms -p 3306:3306 -e MYSQL_ROOT_PASSWORD=pwd -d mysql:8
```

We didn't use an iteractive mode (we didn't use that familiar `-it`), so we should still be in the native OS environment. But if you try

```bash
docker ps
```

You should be able to see your container with the name `ms`. Feel free to close your terminal, we will switch to MySQLWorkbench and jupyter notebook.

### Connection

To confirm that we have started the MySQL database successfully, we build the connection on MySQLWorkbench with:
- Hostname: localhost
- Port: the port number you used in your docker run command
- Username: root
- Connection name: a connection name is required but any string would work

Once you built the connection, you can click on it then MySQLWorkbench will ask for password. That's the password you specified in your docker run command.

As usual, try the `SHOW DATABASES;` command. You will see quite a short list because this is a new and quite empty instance. But as long as you see some databases are returned, you are good to go. Before we can practice any SQL questions, we need to upload data. That's what we are going to do next with a Python code in the Jupyter Notebook, `table_creating_script_generator.ipynb`. Please open it now.

### Uploading Data

The code in `table_creating_script_generator.ipynb` helps to create script that upload data. Run it in MySQLWorkbench then you have the tables you need.

### Some Examples of Window Function

Often the way a window function works is very similar to `GROUP BY`. The main difference is that it doesn't collapse the a group to a single row. To understand it's application, consider the example below:

Date_reported  | Country_code  | Cumulative_cases
-------------- | ------------- | -----------------
2020-02-29     |   CA |  18
2020-03-01     |   CA |  23
2020-03-02     |   CA |  31
2020-01-30     |   US |  10
2020-01-31     |   US |  11
2020-02-01     |   US |  12

Consider if we want to know the new cases everyda, pretending that we don't have it in this table, often we want to create a new column by shifting the Cumulative_cases to get:

Date_reported  | Country_code  | Cumulative_cases  | Cumulative_cases_prev_day
-------------- | ------------- | ----------------- |-----------------
2020-02-29     |   CA |  18 | NA
2020-03-01     |   CA |  23 | 18
2020-03-02     |   CA |  31 | 23
2020-01-30     |   US |  10 | NA
2020-01-31     |   US |  11 | 11
2020-02-01     |   US |  12 | 12

So that the differences between the two columns would be the number of new cases. However, shifting directly  will not result in a table above, insead it would be like:

Date_reported  | Country_code  | Cumulative_cases  | Cumulative_cases_prev_day
-------------- | ------------- | ----------------- |-----------------
2020-02-29     |   CA |  18 | NA
2020-03-01     |   CA |  23 | 18
2020-03-02     |   CA |  31 | 23
2020-01-30     |   US |  10 | 31
2020-01-31     |   US |  11 | 11
2020-02-01     |   US |  12 | 12

and 10 - 31 makes no sense! This motivates a genearl idea of window function - shifting needs to be done within each group/country!

So the syntax for that is:

```SQL
SELECT Date_reported, Country_code, Country, New_cases,Cumulative_cases,
	LAG(Cumulative_cases) OVER (PARTITION BY Country_code ORDER BY Date_reported) AS cases_yesterday
FROM who_data;
```

**Remark**: Here we see that window function won't collapse the observations of the same country into a single row as `GROUP BY`. A sensitive audience might notice that not only to prevent mixing data from different countries when shifting, `ORDER BY` also relies on partitioning. All details aside, often to come up with a right query we need to think about whether the output should be a single row or as many rows as in the original table.


Let's now consider the opposite example, from daily new cases to cumulative cases.

Date_reported  | Country_code  | New_cases
-------------- | ------------- | -----------------
2020-02-29     |   CA |  0
2020-03-01     |   CA |  5
2020-03-02     |   CA |  8
2020-01-30     |   US |  1
2020-01-31     |   US |  1
2020-02-01     |   US |  1

Again, we need to sum up the rows but return a number for each row, that's again a window function case! The query is like:

```SQL
SELECT
	Date_reported, Country_code, Cumulative_cases,
    SUM(New_Cases) OVER (PARTITION BY Country_code ORDER BY Date_reported) AS derived_cumsum,
FROM who_data;
```

### Review Old Tricks

**Exercise**: 
- List the daily growth rate for each country after it reached 1000 daily new cases, if it did already.
- What's the average of the aforementioned daily growth rate for each WHO region.